# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

from entity import Gene
import conf

# Settings

In [3]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses')

In [4]:
INPUT_DATA_DIR = Path(
    conf.RESULTS["DRUG_DISEASE_ANALYSES"],
    "data",
)
display(INPUT_DATA_DIR)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data')

In [5]:
OUTPUT_PREDICTIONS_DIR = Path(conf.RESULTS["DRUG_DISEASE_ANALYSES"], "predictions", "dotprod_neg")
display(OUTPUT_PREDICTIONS_DIR)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg')

# PharmacotherapyDB: load gold standard

## Final

In [6]:
gold_standard = pd.read_pickle(
    Path(conf.RESULTS["DRUG_DISEASE_ANALYSES"], "gold_standard.pkl"),
)

In [7]:
gold_standard.shape

(998, 3)

In [8]:
gold_standard.head()

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [9]:
gold_standard["trait"].unique().shape

(87,)

In [10]:
gold_standard["drug"].unique().shape

(465,)

In [11]:
gold_standard["true_class"].value_counts()

1    755
0    243
Name: true_class, dtype: int64

In [12]:
gold_standard["true_class"].value_counts(normalize=True)

1    0.756513
0    0.243487
Name: true_class, dtype: float64

## Info

In [13]:
# TODO hardcoded
input_file = Path(
    conf.DATA_DIR, "hetionet/pharmacotherapydb-v1.0", "indications.tsv"
).resolve()
display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/hetionet/pharmacotherapydb-v1.0/indications.tsv')

In [14]:
gold_standard_info = pd.read_csv(input_file, sep="\t")

In [15]:
gold_standard_info = gold_standard_info.rename(columns={"drug": "drug_name"})

In [16]:
gold_standard_info.shape

(1388, 7)

In [17]:
gold_standard_info.head()

,doid_id,drugbank_id,disease,drug_name,category,n_curators,n_resources
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,DM,2,1
1,DOID:10652,DB00674,Alzheimer's disease,Galantamine,DM,1,4
2,DOID:10652,DB01043,Alzheimer's disease,Memantine,DM,1,3
3,DOID:10652,DB00989,Alzheimer's disease,Rivastigmine,DM,1,3
4,DOID:10652,DB00245,Alzheimer's disease,Benzatropine,SYM,3,1


In [18]:
gold_standard_info = gold_standard.set_index(
    ["trait", "drug"]
).join(
    gold_standard_info.rename(
        columns={"doid_id": "trait", "drugbank_id": "drug"}
    ).set_index(["trait", "drug"])).reset_index()

In [19]:
gold_standard_info.shape

(998, 8)

In [20]:
gold_standard_info.head()

,trait,drug,true_class,disease,drug_name,category,n_curators,n_resources
0,DOID:10652,DB00843,1,Alzheimer's disease,Donepezil,DM,2,1
1,DOID:10652,DB00674,1,Alzheimer's disease,Galantamine,DM,1,4
2,DOID:10652,DB01043,1,Alzheimer's disease,Memantine,DM,1,3
3,DOID:10652,DB00989,1,Alzheimer's disease,Rivastigmine,DM,1,3
4,DOID:10652,DB00810,0,Alzheimer's disease,Biperiden,NOT,2,1


# LINCS data

In [21]:
input_file = Path(INPUT_DATA_DIR, "raw", "lincs-data.pkl").resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw/lincs-data.pkl')

In [22]:
lincs_data = pd.read_pickle(input_file).T.rename(columns=Gene.GENE_ID_TO_NAME_MAP)

In [23]:
display(lincs_data.shape)

(1170, 7120)

In [24]:
display(lincs_data.head())

,ADA,CDH2,AKT3,MED6,ACOT8,ABI1,GNPDA1,CDH3,TANK,TOPORS-AS1,...,RBX1,CDC42,THOC1,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,PTBP3,CASP8AP2
perturbagen,,,,,,,,,,,,,,,,,,,,,
DB00014,-1.001,1.146,-0.693,-0.037,0.162,-0.380,0.649,-1.952,0.294,0.274,...,-0.592,0.178,0.270,-0.013,0.351,-0.128,0.274,-0.734,-0.048,0.036
DB00091,-1.835,-1.863,1.694,0.383,-0.899,0.628,-4.878,2.527,1.709,-0.981,...,-0.668,-2.795,-0.333,-0.027,0.578,6.926,-1.875,4.382,0.266,-4.995
DB00121,1.391,0.011,-0.804,0.269,0.105,-0.588,-1.899,0.306,-1.178,0.120,...,-0.962,0.450,-0.999,1.358,-1.476,0.423,-1.356,-1.897,-0.299,-0.732
DB00130,1.132,-1.020,-0.164,-0.997,-0.090,0.195,-2.341,0.494,-0.813,-1.140,...,-0.553,-0.528,0.308,0.534,-0.320,3.001,-0.414,0.134,0.147,2.158
DB00131,0.257,1.143,1.145,0.185,-1.291,-0.457,0.038,1.442,-1.692,0.593,...,-0.556,-0.600,0.092,-0.564,0.174,-1.203,-1.080,-0.575,-0.792,-0.095


In [25]:
lincs_data.index.is_unique

True

In [26]:
lincs_data.columns.is_unique

True

# LINCS projection

In [27]:
input_file = Path(INPUT_DATA_DIR, "proj", "lincs-projection.pkl").resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj/lincs-projection.pkl')

In [28]:
lincs_projection = pd.read_pickle(input_file).T

In [29]:
display(lincs_projection.shape)

(1170, 987)

In [30]:
display(lincs_projection.head())

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
perturbagen,,,,,,,,,,,,,,,,,,,,,
DB00014,0.036115,0.012281,-0.005533,-0.004151,-0.015156,-0.010861,0.008181,-0.008148,0.004948,-0.009869,...,-0.003139,0.003546,-0.006242,0.002449,-0.004391,-0.004334,-0.019902,0.004292,-0.010378,0.002000
DB00091,-0.358610,0.009738,-0.059174,-0.032884,-0.005276,0.063369,-0.079522,-0.018980,-0.025782,0.021540,...,-0.019892,0.003610,-0.016165,0.008686,0.065335,-0.042051,0.065516,0.016261,0.056314,-0.051035
DB00121,0.091067,0.004650,0.013454,0.005934,0.025747,0.004324,-0.001865,-0.019613,0.051503,0.013032,...,0.001629,-0.012565,0.009613,-0.014404,-0.009188,0.006544,0.047466,-0.016220,-0.018844,0.034675
DB00130,0.008923,-0.006554,0.008906,-0.003991,0.000346,0.008212,0.001242,-0.012771,0.014221,0.008269,...,-0.026139,-0.002977,0.021969,0.001204,-0.012731,-0.000076,-0.029127,-0.022679,0.016179,-0.003029
DB00131,0.046469,0.021758,-0.004939,-0.028524,-0.013932,0.012231,-0.001325,0.003787,0.035189,-0.004377,...,0.017373,0.005304,-0.001649,0.004017,-0.000622,-0.006477,0.021663,0.000988,0.002038,-0.011747


# MultiPLIER Z

In [31]:
multiplier_z = pd.read_pickle(conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"])

In [32]:
multiplier_z.shape

(6750, 987)

In [33]:
multiplier_z.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
GAS6,0.000000,0.0,0.039438,0.0,0.050476,0.000000,0.0,0.000000,0.590949,0.000000,...,0.050125,0.00000,0.033407,0.000000,0.000000,0.005963,0.347362,0.0,0.000000,0.000000
MMP14,0.000000,0.0,0.000000,0.0,0.070072,0.000000,0.0,0.004904,1.720179,2.423595,...,0.000000,0.00000,0.001007,0.000000,0.035747,0.000000,0.000000,0.0,0.014978,0.000000
DSP,0.000000,0.0,0.000000,0.0,0.000000,0.041697,0.0,0.005718,0.000000,0.000000,...,0.020853,0.00000,0.000000,0.000000,0.000000,0.005774,0.000000,0.0,0.000000,0.416405
MARCKSL1,0.305212,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.161843,0.149471,...,0.027134,0.05272,0.000000,0.030189,0.060884,0.000000,0.000000,0.0,0.000000,0.448480
SPARC,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.014014,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.067779,0.0,0.122417,0.062665


# Prediction results

## Full

In [34]:
output_file = Path(OUTPUT_DIR, "predictions", "predictions_results.pkl").resolve()
display(output_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/predictions_results.pkl')

In [35]:
predictions = pd.read_pickle(output_file)

In [36]:
predictions.shape

(316540, 8)

In [37]:
predictions.head()

,trait,drug,score,true_class,method,n_top_genes,data,tissue
0,DOID:0050741,DB00215,146.0,1,Gene-based,-1.0,spredixcan-mashr-zscores-Adipose_Subcutaneous-...,Adipose_Subcutaneous
1,DOID:0050741,DB00704,465.0,1,Gene-based,-1.0,spredixcan-mashr-zscores-Adipose_Subcutaneous-...,Adipose_Subcutaneous
2,DOID:0050741,DB00822,530.0,1,Gene-based,-1.0,spredixcan-mashr-zscores-Adipose_Subcutaneous-...,Adipose_Subcutaneous
3,DOID:10283,DB00014,124.0,1,Gene-based,-1.0,spredixcan-mashr-zscores-Adipose_Subcutaneous-...,Adipose_Subcutaneous
4,DOID:10283,DB00175,309.0,0,Gene-based,-1.0,spredixcan-mashr-zscores-Adipose_Subcutaneous-...,Adipose_Subcutaneous


In [38]:
def _reduce_mean(x):
    return pd.Series(
        {
            "score": x["score"].mean(),
            "true_class": x["true_class"].unique()[0]
            #             if x["true_class"].unique().shape[0] == 1
            #             else None,
        }
    )

In [39]:
predictions_by_tissue = (
    predictions.groupby(["trait", "drug", "method", "tissue"])
    .apply(_reduce_mean)
    .dropna()
    .sort_index()
    .reset_index()
)

In [40]:
predictions_by_tissue.head()

,trait,drug,method,tissue,score,true_class
0,DOID:0050741,DB00215,Gene-based,Adipose_Subcutaneous,92.0,1.0
1,DOID:0050741,DB00215,Gene-based,Adipose_Visceral_Omentum,114.2,1.0
2,DOID:0050741,DB00215,Gene-based,Adrenal_Gland,243.2,1.0
3,DOID:0050741,DB00215,Gene-based,Artery_Aorta,163.4,1.0
4,DOID:0050741,DB00215,Gene-based,Artery_Coronary,116.6,1.0


## Aggregated

In [41]:
output_file = Path(
    OUTPUT_DIR, "predictions", "predictions_results_aggregated.pkl"
).resolve()
display(output_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/predictions_results_aggregated.pkl')

In [42]:
predictions_avg = pd.read_pickle(output_file)

In [43]:
predictions_avg.shape

(1292, 5)

In [44]:
predictions_avg.head()

,trait,drug,method,score,true_class
0,DOID:0050741,DB00215,Gene-based,460.0,1.0
1,DOID:0050741,DB00215,Module-based,437.8,1.0
2,DOID:0050741,DB00704,Gene-based,544.4,1.0
3,DOID:0050741,DB00704,Module-based,562.8,1.0
4,DOID:0050741,DB00822,Gene-based,598.2,1.0


# Merge

In [45]:
pharmadb_predictions = pd.merge(
    gold_standard_info,
    predictions_avg,
#     left_on=["doid_id", "drugbank_id"],
    on=["trait", "drug"],
    how="inner",
)

In [46]:
pharmadb_predictions

,trait,drug,true_class_x,disease,drug_name,category,n_curators,n_resources,method,score,true_class_y
0,DOID:10652,DB00843,1,Alzheimer's disease,Donepezil,DM,2,1,Gene-based,345.0,1.0
1,DOID:10652,DB00843,1,Alzheimer's disease,Donepezil,DM,2,1,Module-based,475.2,1.0
2,DOID:10652,DB00674,1,Alzheimer's disease,Galantamine,DM,1,4,Gene-based,505.4,1.0
3,DOID:10652,DB00674,1,Alzheimer's disease,Galantamine,DM,1,4,Module-based,508.0,1.0
4,DOID:10652,DB01043,1,Alzheimer's disease,Memantine,DM,1,3,Gene-based,484.6,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1287,DOID:363,DB00917,0,uterine cancer,Dinoprostone,NOT,3,1,Module-based,476.0,0.0
1288,DOID:363,DB00783,0,uterine cancer,Estradiol,NOT,2,1,Gene-based,546.2,0.0
1289,DOID:363,DB00783,0,uterine cancer,Estradiol,NOT,2,1,Module-based,603.0,0.0
1290,DOID:12306,DB00553,1,vitiligo,Methoxsalen,DM,3,1,Gene-based,513.4,1.0


In [47]:
pharmadb_predictions = pharmadb_predictions[
    ["trait", "drug", "disease", "drug_name", "method", "score", "true_class_x"]
].rename(columns={"true_class_x": "true_class", "drug_x": "drug"})

In [48]:
display(pharmadb_predictions.shape)
assert pharmadb_predictions.shape[0] == predictions_avg.shape[0]

(1292, 7)

In [49]:
pharmadb_predictions.head()

,trait,drug,disease,drug_name,method,score,true_class
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,Gene-based,345.0,1
1,DOID:10652,DB00843,Alzheimer's disease,Donepezil,Module-based,475.2,1
2,DOID:10652,DB00674,Alzheimer's disease,Galantamine,Gene-based,505.4,1
3,DOID:10652,DB00674,Alzheimer's disease,Galantamine,Module-based,508.0,1
4,DOID:10652,DB01043,Alzheimer's disease,Memantine,Gene-based,484.6,1


In [50]:
pharmadb_predictions["trait"].unique().shape

(53,)

In [51]:
pharmadb_predictions["drug"].unique().shape

(322,)

In [52]:
data_stats = pharmadb_predictions.groupby("method")["score"].describe()
display(data_stats)

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Gene-based,646.0,520.671207,89.368962,205.6,464.5,535.3,591.8,646.0
Module-based,646.0,530.429721,73.472425,220.4,484.1,535.0,587.9,646.0


In [53]:
# Standardize scores by method
def _standardize(x):
#     return (x["score"] - data_stats.loc[x["method"], "min"]) / (
#         data_stats.loc[x["method"], "max"] - data_stats.loc[x["method"], "min"]
#     )
    return (x["score"] - data_stats.loc[x["method"], "mean"]) / data_stats.loc[
        x["method"], "std"
    ]

In [54]:
pharmadb_predictions = pharmadb_predictions.assign(
    score_std=pharmadb_predictions.apply(_standardize, axis=1)
)

In [55]:
pharmadb_predictions

,trait,drug,disease,drug_name,method,score,true_class,score_std
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,Gene-based,345.0,1,-1.965685
1,DOID:10652,DB00843,Alzheimer's disease,Donepezil,Module-based,475.2,1,-0.751707
2,DOID:10652,DB00674,Alzheimer's disease,Galantamine,Gene-based,505.4,1,-0.170878
3,DOID:10652,DB00674,Alzheimer's disease,Galantamine,Module-based,508.0,1,-0.305281
4,DOID:10652,DB01043,Alzheimer's disease,Memantine,Gene-based,484.6,1,-0.403621
...,...,...,...,...,...,...,...,...
1287,DOID:363,DB00917,uterine cancer,Dinoprostone,Module-based,476.0,0,-0.740818
1288,DOID:363,DB00783,uterine cancer,Estradiol,Gene-based,546.2,0,0.285656
1289,DOID:363,DB00783,uterine cancer,Estradiol,Module-based,603.0,0,0.987721
1290,DOID:12306,DB00553,vitiligo,Methoxsalen,Gene-based,513.4,1,-0.081362


In [56]:
_tmp = pharmadb_predictions.groupby("method")[["score", "score_std"]].describe()
display(_tmp)

score                                                            \
              count        mean        std    min    25%    50%    75%    max   
method                                                                          
Gene-based    646.0  520.671207  89.368962  205.6  464.5  535.3  591.8  646.0   
Module-based  646.0  530.429721  73.472425  220.4  484.1  535.0  587.9  646.0   

             score_std                                                   \
                 count          mean  std       min       25%       50%   
method                                                                    
Gene-based       646.0  9.459238e-16  1.0 -3.525510 -0.628531  0.163690   
Module-based     646.0  2.034836e-16  1.0 -4.219675 -0.630573  0.062204   

                                  
                   75%       max  
method                            
Gene-based    0.795900  1.402375  
Module-based  0.782202  1.572975

In [57]:
_tmp0 = pharmadb_predictions[(pharmadb_predictions["method"] == "Gene-based")][
    ["score", "score_std"]
]

In [58]:
assert all(_tmp0.corr() > 0.99999)

In [59]:
_tmp0 = pharmadb_predictions[(pharmadb_predictions["method"] == "Module-based")][
    ["score", "score_std"]
]

In [60]:
assert all(_tmp0.corr() > 0.99999)

# List diseases

In [61]:
pharmadb_predictions["disease"].unique()

array(["Alzheimer's disease", "Crohn's disease", "Parkinson's disease",
       'alcohol dependence', 'allergic rhinitis', 'anemia',
       'ankylosing spondylitis', 'asthma', 'atherosclerosis',
       'bone cancer', 'brain cancer', 'breast cancer', 'cervical cancer',
       'chronic kidney failure', 'chronic obstructive pulmonary disease',
       'coronary artery disease', 'epilepsy syndrome',
       'esophageal cancer', 'gestational diabetes', 'glaucoma', 'gout',
       'hematologic cancer', 'hypertension', 'hypothyroidism',
       'kidney cancer', 'liver cancer', 'lung cancer', 'malaria',
       'melanoma', 'migraine', 'multiple sclerosis', 'nephrolithiasis',
       'obesity', 'osteoarthritis', 'osteoporosis', 'pancreatic cancer',
       'pancreatitis', 'periodontitis', 'prostate cancer', 'psoriasis',
       'psoriatic arthritis', 'rheumatoid arthritis', 'skin cancer',
       'stomach cancer', 'systemic lupus erythematosus',
       'testicular cancer', 'thyroid cancer', 'type 1 diabe

# Looks for differences in scores of both methods

In [62]:
np.all(pharmadb_predictions.groupby(["trait", "drug"]).count() == 2)

True

In [63]:
def _compare(x):
    assert x.shape[0] == 2
    x_sign = np.sign(x["score_std"].values)
    x0 = x.iloc[0]["score_std"]
    x1 = x.iloc[1]["score_std"]
    
    return pd.Series({
        "different_sign": x_sign[0] != x_sign[1],
        "score_difference": np.abs(x0 - x1)
    })

In [64]:
pharmadb_predictions = pharmadb_predictions.set_index(["trait", "drug"]).join(
    pharmadb_predictions.groupby(["trait", "drug"]).apply(_compare)
)

In [65]:
pharmadb_predictions.head()

disease   drug_name        method  score  \
trait        drug                                                           
DOID:0050741 DB00215  alcohol dependence  Citalopram    Gene-based  460.0   
             DB00215  alcohol dependence  Citalopram  Module-based  437.8   
             DB00704  alcohol dependence  Naltrexone    Gene-based  544.4   
             DB00704  alcohol dependence  Naltrexone  Module-based  562.8   
             DB00822  alcohol dependence  Disulfiram    Gene-based  598.2   

                      true_class  score_std  different_sign  score_difference  
trait        drug                                                              
DOID:0050741 DB00215           1  -0.678885           False          0.581857  
             DB00215           1  -1.260741           False          0.581857  
             DB00704           1   0.265515           False          0.175062  
             DB00704           1   0.440577           False          0.175062  
             DB00822           1   0.867514           False          0.057599

In [66]:
def find_differences(trait_name):
    with pd.option_context(
        "display.max_rows", None, "display.max_columns", None, "max_colwidth", None
    ):
        _tmp = (
            pharmadb_predictions[
                (pharmadb_predictions["disease"] == trait_name)
                & (pharmadb_predictions["different_sign"])
            ]
            .sort_values(["score_difference", "drug_name", "method"], ascending=[False, False, False])
        )
        display(_tmp)

## any disease

In [67]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "max_colwidth", None
):
    _tmp = (
        pharmadb_predictions[
            (pharmadb_predictions["different_sign"])
            & (~pharmadb_predictions["disease"].str.contains("cancer")) # avoid cancer
        ]
        .sort_values(["score_difference", "drug_name", "method"], ascending=[False, False, False])
    )
    display(_tmp.head(100))

disease            drug_name  \
trait      drug                                                         
DOID:10763 DB01136                  hypertension           Carvedilol   
           DB01136                  hypertension           Carvedilol   
DOID:9008  DB01234           psoriatic arthritis        Dexamethasone   
           DB01234           psoriatic arthritis        Dexamethasone   
           DB00443           psoriatic arthritis        Betamethasone   
           DB00443           psoriatic arthritis        Betamethasone   
DOID:13189 DB01394                          gout           Colchicine   
           DB01394                          gout           Colchicine   
DOID:7147  DB00620        ankylosing spondylitis        Triamcinolone   
           DB00620        ankylosing spondylitis        Triamcinolone   
DOID:8577  DB01014            ulcerative colitis          Balsalazide   
           DB01014            ulcerative colitis          Balsalazide   
DOID:10763 DB00820                  hypertension            Tadalafil   
           DB00820                  hypertension            Tadalafil   
DOID:2841  DB00938                        asthma           Salmeterol   
           DB00938                        asthma           Salmeterol   
DOID:4989  DB00424                  pancreatitis          Hyoscyamine   
           DB00424                  pancreatitis          Hyoscyamine   
           DB00572                  pancreatitis             Atropine   
           DB00572                  pancreatitis             Atropine   
DOID:10763 DB01001                  hypertension           Salbutamol   
           DB01001                  hypertension           Salbutamol   
           DB00758                  hypertension          Clopidogrel   
           DB00758                  hypertension          Clopidogrel   
           DB00598                  hypertension            Labetalol   
           DB00598                  hypertension            Labetalol   
DOID:12365 DB00205                       malaria        Pyrimethamine   
           DB00205                       malaria        Pyrimethamine   
DOID:9352  DB00222      type 2 diabetes mellitus          Glimepiride   
           DB00222      type 2 diabetes mellitus          Glimepiride   
DOID:4481  DB01234             allergic rhinitis        Dexamethasone   
           DB01234             allergic rhinitis        Dexamethasone   
           DB00443             allergic rhinitis        Betamethasone   
           DB00443             allergic rhinitis        Betamethasone   
DOID:1936  DB00227               atherosclerosis           Lovastatin   
           DB00227               atherosclerosis           Lovastatin   
DOID:4481  DB00741             allergic rhinitis       Hydrocortisone   
           DB00741             allergic rhinitis       Hydrocortisone   
DOID:9074  DB00959  systemic lupus erythematosus   Methylprednisolone   
           DB00959  systemic lupus erythematosus   Methylprednisolone   
DOID:585   DB00999               nephrolithiasis  Hydrochlorothiazide   
           DB00999               nephrolithiasis  Hydrochlorothiazide   
DOID:1936  DB00973               atherosclerosis            Ezetimibe   
           DB00973               atherosclerosis            Ezetimibe   
DOID:10763 DB00521                  hypertension            Carteolol   
           DB00521                  hypertension            Carteolol   
DOID:9352  DB01200      type 2 diabetes mellitus        Bromocriptine   
           DB01200      type 2 diabetes mellitus        Bromocriptine   
DOID:8778  DB01014               Crohn's disease          Balsalazide   
           DB01014               Crohn's disease          Balsalazide   
DOID:10763 DB01194                  hypertension         Brinzolamide   
           DB01194                  hypertension         Brinzolamide   
DOID:1826  DB00909             epilepsy syndrome           Zonisamide   
           DB00909        

## coronary artery disease

In [68]:
find_differences("coronary artery disease")

disease    drug_name        method  score  \
trait     drug                                                                 
DOID:3393 DB00177  coronary artery disease    Valsartan  Module-based  431.0   
          DB00177  coronary artery disease    Valsartan    Gene-based  531.6   
          DB00700  coronary artery disease   Eplerenone  Module-based  538.6   
          DB00700  coronary artery disease   Eplerenone    Gene-based  428.2   
          DB00627  coronary artery disease       Niacin  Module-based  601.0   
          DB00627  coronary artery disease       Niacin    Gene-based  505.6   
          DB01197  coronary artery disease    Captopril  Module-based  580.2   
          DB01197  coronary artery disease    Captopril    Gene-based  488.2   
          DB00988  coronary artery disease     Dopamine  Module-based  527.6   
          DB00988  coronary artery disease     Dopamine    Gene-based  560.6   
          DB00790  coronary artery disease  Perindopril  Module-based  555.8   
          DB00790  coronary artery disease  Perindopril    Gene-based  519.8   
          DB00175  coronary artery disease  Pravastatin  Module-based  547.0   
          DB00175  coronary artery disease  Pravastatin    Gene-based  515.8   
          DB01583  coronary artery disease      Liotrix  Module-based  534.0   
          DB01583  coronary artery disease      Liotrix    Gene-based  501.8   

                   true_class  score_std  different_sign  score_difference  
trait     drug                                                              
DOID:3393 DB00177           1  -1.353293            True          1.475582  
          DB00177           1   0.122288            True          1.475582  
          DB00700           1   0.111202            True          1.145915  
          DB00700           1  -1.034713            True          1.145915  
          DB00627           1   0.960500            True          1.129140  
          DB00627           1  -0.168640            True          1.129140  
          DB01197           1   0.677401            True          1.040740  
          DB01197           1  -0.363339            True          1.040740  
          DB00988           0  -0.038514            True          0.485300  
          DB00988           0   0.446786            True          0.485300  
          DB00790           1   0.345303            True          0.355052  
          DB00790           1  -0.009748            True          0.355052  
          DB00175           1   0.225531            True          0.280037  
          DB00175           1  -0.054507            True          0.280037  
          DB01583           0   0.048593            True          0.259754  
          DB01583           0  -0.211161            True          0.259754

## atherosclerosis

In [69]:
find_differences("atherosclerosis")

disease    drug_name        method  score  \
trait     drug                                                         
DOID:1936 DB00227  atherosclerosis   Lovastatin  Module-based  439.0   
          DB00227  atherosclerosis   Lovastatin    Gene-based  563.4   
          DB00973  atherosclerosis    Ezetimibe  Module-based  420.2   
          DB00973  atherosclerosis    Ezetimibe    Gene-based  533.0   
          DB00758  atherosclerosis  Clopidogrel  Module-based  503.2   
          DB00758  atherosclerosis  Clopidogrel    Gene-based  582.2   
          DB00627  atherosclerosis       Niacin  Module-based  568.6   
          DB00627  atherosclerosis       Niacin    Gene-based  512.6   

                   true_class  score_std  different_sign  score_difference  
trait     drug                                                              
DOID:1936 DB00227           1  -1.244409            True          1.722525  
          DB00227           1   0.478117            True          1.722525  
          DB00973           1  -1.500287            True          1.638241  
          DB00973           1   0.137954            True          1.638241  
          DB00758           0  -0.370611            True          1.059092  
          DB00758           0   0.688481            True          1.059092  
          DB00627           1   0.519518            True          0.609832  
          DB00627           1  -0.090313            True          0.609832

# Niacin and vascular diseases

In [177]:
_doid = "DOID:1936"
_phenomexcan_traits = ["I70-Diagnoses_main_ICD10_I70_Atherosclerosis"]

# _doid = "DOID:3393"
# _phenomexcan_traits = [
#     "I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease",
# #     "CARDIoGRAM_C4D_CAD_ADDITIVE"
# ]

_drug_id = "DB00627"
_drug_name = "Niacin"

In [178]:
pharmadb_predictions[pharmadb_predictions["drug_name"] == _drug_name].sort_values(["disease", "method"])

disease drug_name        method  score  \
trait     drug                                                              
DOID:1936 DB00627          atherosclerosis    Niacin    Gene-based  512.6   
          DB00627          atherosclerosis    Niacin  Module-based  568.6   
DOID:3393 DB00627  coronary artery disease    Niacin    Gene-based  505.6   
          DB00627  coronary artery disease    Niacin  Module-based  601.0   
DOID:4989 DB00627             pancreatitis    Niacin    Gene-based  485.2   
          DB00627             pancreatitis    Niacin  Module-based  518.8   

                   true_class  score_std  different_sign  score_difference  
trait     drug                                                              
DOID:1936 DB00627           1  -0.090313            True          0.609832  
          DB00627           1   0.519518            True          0.609832  
DOID:3393 DB00627           1  -0.168640            True          1.129140  
          DB00627           1   0.960500            True          1.129140  
DOID:4989 DB00627           0  -0.396907           False          0.238621  
          DB00627           0  -0.158287           False          0.238621

In [179]:
predictions_avg[
    (predictions_avg["trait"] == _doid)
    & (predictions_avg["drug"] == _drug_id)
]

,trait,drug,method,score,true_class
588,DOID:1936,DB00627,Gene-based,512.6,1.0
589,DOID:1936,DB00627,Module-based,568.6,1.0


In [310]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "max_colwidth", None
):
    _tmp = predictions_by_tissue[
        (predictions_by_tissue["trait"] == _doid)
        & (predictions_by_tissue["drug"] == _drug_id)
    ].groupby("method")["score"].idxmax()
    display(predictions_by_tissue.loc[_tmp.values])
    
    _tmp = predictions_by_tissue.loc[_tmp.values][["method", "tissue"]].set_index("method")
    _gene_based_tissue = _tmp.loc["Gene-based", "tissue"]
    display(_gene_based_tissue)
    
    _module_tissue = _tmp.loc["Module-based", "tissue"]
    display(_module_tissue)

,trait,drug,method,tissue,score,true_class
28856,DOID:1936,DB00627,Gene-based,Testis,512.6,1.0
28881,DOID:1936,DB00627,Module-based,Cells_Cultured_fibroblasts,568.6,1.0


'Testis'

'Cells_Cultured_fibroblasts'

In [311]:
# predictions_by_tissue[
#     (predictions_by_tissue["trait"] == _doid)
#     & (predictions_by_tissue["drug"] == _drug_id)
# ].sort_values("score", ascending=False)

In [312]:
# _module_tissue = "Adipose_Subcutaneous"

### Gene module-based - LVs driving association

In [313]:
_module_tissue_data_raw = pd.read_pickle(INPUT_DATA_DIR / "raw" / f"spredixcan-mashr-zscores-{_module_tissue}-data.pkl")
_module_tissue_data_raw = _module_tissue_data_raw.rename(index=Gene.GENE_ID_TO_NAME_MAP)

display(_module_tissue_data_raw.shape)
display(_module_tissue_data_raw.head())

(13395, 4091)

,I9_PHLETHROMBDVTLOW-DVT_of_lower_extremities,I71-Diagnoses_main_ICD10_I71_Aortic_aneurysm_and_dissection,G62-Diagnoses_main_ICD10_G62_Other_polyneuropathies,2395_4-Hairbalding_pattern_Pattern_4,20003_1141168590-Treatmentmedication_code_pariet_10mg_ec_tablet,4056_raw-Age_stroke_diagnosed,M13_FOREIGNBODY-Residual_foreign_body_in_soft_tissue,20002_1494-Noncancer_illness_code_selfreported_varicose_veins,I85-Diagnoses_main_ICD10_I85_Oesophageal_varices,6158_4-Why_reduced_smoking_Financial_reasons,...,41215_0-Detention_categories_Informal_not_formally_detained,M76-Diagnoses_main_ICD10_M76_Enthesopathies_of_lower_limb_excluding_foot,22601_35633208-Job_coding_vocational_or_industrial_trainerinstructor,H7_RETINALDETACH-Retinal_detachments_and_breaks,20003_1141172918-Treatmentmedication_code_celluvisc_1_singleuse_eye_drops,22601_35643209-Job_coding_careers_adviser_or_vocational_guidance_specialist_placement_advisor,20002_1537-Noncancer_illness_code_selfreported_joint_pain,20514-Recent_lack_of_interest_or_pleasure_in_doing_things,22601_53192534-Job_coding_builder_building_contractor,30280_raw-Immature_reticulocyte_fraction
gene_id,,,,,,,,,,,,,,,,,,,,,
SCYL3,1.542728,0.374937,-1.180026,3.617304,-1.436877,-1.150046,-0.671733,0.681798,1.653683,1.580276,...,0.100557,1.150549,-0.099058,-0.522253,-0.875756,-0.323892,-1.355607,0.546768,-2.618420,-0.003352
CFH,2.221978,-0.029211,-0.082311,-0.707300,0.860528,-0.435167,0.210884,1.454587,1.331105,-0.155145,...,3.271953,-0.805236,-1.050279,-0.311062,-0.266324,-0.961981,1.025476,1.432934,1.262197,-0.042457
FUCA2,-1.290055,0.997560,-0.608218,-0.704553,-1.278815,-1.201553,0.817245,0.653960,0.318084,-0.600657,...,-2.057690,0.036173,0.498194,0.824913,0.674138,0.121689,-0.093050,2.578375,-0.591618,-1.505238
GCLC,1.039654,0.020216,-0.175122,0.427606,-0.152778,2.209256,0.484214,1.482017,0.495467,-1.217398,...,0.538651,-0.282506,-0.253741,-0.948048,-0.561655,0.990805,1.286077,1.382912,0.836434,-0.015618
STPG1,-0.848960,0.083508,-0.112559,-3.302466,-0.956408,-0.402499,-0.236055,-0.007223,1.315015,0.550110,...,2.092645,0.320062,0.584716,-0.415151,0.542228,-0.520030,0.912408,0.971265,1.032357,-1.008020


In [314]:
_module_tissue_data_raw.index.is_unique

False

In [315]:
_module_tissue_data_raw.index[_module_tissue_data_raw.index.duplicated(keep="first")]

Index(['SPATA13', 'LINC00484'], dtype='object', name='gene_id')

In [316]:
_module_tissue_data_raw = _module_tissue_data_raw.loc[
    ~_module_tissue_data_raw.index.duplicated(keep="first")
]

In [317]:
_module_tissue_data_raw.shape

(13393, 4091)

In [318]:
_module_tissue_data_raw.index.is_unique

True

In [319]:
_module_tissue_data = pd.read_pickle(INPUT_DATA_DIR / "proj" / f"spredixcan-mashr-zscores-{_module_tissue}-projection.pkl")
display(_module_tissue_data.head())

,I9_PHLETHROMBDVTLOW-DVT_of_lower_extremities,I71-Diagnoses_main_ICD10_I71_Aortic_aneurysm_and_dissection,G62-Diagnoses_main_ICD10_G62_Other_polyneuropathies,2395_4-Hairbalding_pattern_Pattern_4,20003_1141168590-Treatmentmedication_code_pariet_10mg_ec_tablet,4056_raw-Age_stroke_diagnosed,M13_FOREIGNBODY-Residual_foreign_body_in_soft_tissue,20002_1494-Noncancer_illness_code_selfreported_varicose_veins,I85-Diagnoses_main_ICD10_I85_Oesophageal_varices,6158_4-Why_reduced_smoking_Financial_reasons,...,41215_0-Detention_categories_Informal_not_formally_detained,M76-Diagnoses_main_ICD10_M76_Enthesopathies_of_lower_limb_excluding_foot,22601_35633208-Job_coding_vocational_or_industrial_trainerinstructor,H7_RETINALDETACH-Retinal_detachments_and_breaks,20003_1141172918-Treatmentmedication_code_celluvisc_1_singleuse_eye_drops,22601_35643209-Job_coding_careers_adviser_or_vocational_guidance_specialist_placement_advisor,20002_1537-Noncancer_illness_code_selfreported_joint_pain,20514-Recent_lack_of_interest_or_pleasure_in_doing_things,22601_53192534-Job_coding_builder_building_contractor,30280_raw-Immature_reticulocyte_fraction
LV1,-0.019600,0.037370,0.015432,0.103213,-0.043647,0.017953,-0.017080,-0.040636,0.034560,-0.013533,...,0.053705,-0.021551,0.015006,0.020924,0.063630,0.014546,0.031569,0.025669,-0.032848,0.050867
LV2,0.045221,0.015686,-0.018787,0.003183,0.015054,0.006919,-0.032647,0.034986,-0.004918,0.003432,...,0.012654,-0.002942,-0.037361,0.023963,0.001036,-0.009236,-0.047476,-0.033387,0.019026,0.000041
LV3,-0.012891,0.002663,0.045862,-0.024472,0.020005,-0.020490,-0.027314,-0.024277,0.035376,0.010420,...,-0.015451,0.005034,-0.006526,0.024579,0.035953,-0.001680,0.007341,-0.006215,-0.022623,-0.051306
LV4,-0.012022,0.002787,0.006922,-0.038151,0.034503,0.009894,0.018250,0.019581,0.020272,-0.023895,...,-0.022380,-0.035518,-0.028791,-0.021367,-0.018474,0.012211,-0.003103,-0.011157,-0.001598,0.056887
LV5,-0.010747,0.000051,-0.016533,0.035076,0.002675,0.011499,0.011403,0.012799,0.009265,0.035350,...,-0.011572,-0.016130,-0.011499,-0.027826,-0.012679,0.037951,0.067566,-0.046590,-0.018380,0.004543


In [320]:
_all_dfs = []

for ntc in (None, 5, 10, 25, 50):
    _prefix = "all_genes"
    if ntc is not None:
        _prefix = f"top_{ntc}_genes"
    
    _df = pd.read_hdf(
        OUTPUT_PREDICTIONS_DIR / f"spredixcan-mashr-zscores-{_module_tissue}-projection-{_prefix}-prediction_scores.h5",
        key="prediction"
    )
    
    _score_col_name = f"score_{_prefix}"
    _df = pd.merge(
        _df, gold_standard, on=["trait", "drug"], how="inner"
    ).drop(columns=["true_class"]).set_index(["trait", "drug"]).rename(columns={"score": _score_col_name})
    
    _df[_score_col_name] = _df[_score_col_name].rank()
    
    _all_dfs.append(_df)

In [321]:
_df

score_top_50_genes
trait        drug                       
DOID:0050741 DB00215               100.0
             DB00704               420.0
             DB00822               134.0
DOID:10283   DB00014                89.0
             DB00175               146.0
...                                  ...
DOID:9970    DB00397               373.0
             DB00501               323.0
             DB01083               490.0
             DB01105               219.0
             DB01156               599.0

[646 rows x 1 columns]

In [322]:
_predictions_tissue = pd.concat(_all_dfs, axis=1, join="inner")#.mean(axis=1)

In [323]:
_predictions_tissue.shape

(646, 5)

In [324]:
_predictions_tissue.head()

score_all_genes  score_top_5_genes  score_top_10_genes  \
trait        drug                                                              
DOID:0050741 DB00215            281.0              481.0               186.0   
             DB00704            505.0              559.0               523.0   
             DB00822            278.0              329.0               397.0   
DOID:10283   DB00014            115.0               77.0               148.0   
             DB00175            490.0              157.0               285.0   

                      score_top_25_genes  score_top_50_genes  
trait        drug                                             
DOID:0050741 DB00215                83.0               100.0  
             DB00704               533.0               420.0  
             DB00822                61.0               134.0  
DOID:10283   DB00014               190.0                89.0  
             DB00175               112.0               146.0

In [325]:
_predictions_tissue = _predictions_tissue.mean(axis=1)

In [326]:
display(_predictions_tissue.shape)
display(_predictions_tissue.head())

assert _df.shape[0] == _predictions_tissue.shape[0]
assert not _predictions_tissue.isna().any().any()

(646,)

trait         drug   
DOID:0050741  DB00215    226.2
              DB00704    508.0
              DB00822    239.8
DOID:10283    DB00014    123.8
              DB00175    238.0
dtype: float64

In [327]:
_predictions_tissue.loc[_doid, _drug_id]# .query(f"(trait == '{_doid}') and (drug == '{_drug_id}')")

568.6

In [328]:
# FIXME: add assert here to make sure this number above is the same in predictions_avg

In [329]:
# _traits_proj = pd.read_pickle(INPUT_DATA_DIR / "proj" / f"spredixcan-mashr-zscores-{_module_tissue}-projection.pkl")

_module_tissue_data = _module_tissue_data[
    _phenomexcan_traits
#     "C62-Diagnoses_main_ICD10_C62_Malignant_neoplasm_of_testis",
#     "I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease",
]

# _traits_proj = _traits_proj.apply(
#     lambda x: _zero_nontop_genes(x, 50, use_abs=True)
# )

In [330]:
_module_tissue_data.head()

,I70-Diagnoses_main_ICD10_I70_Atherosclerosis
LV1,0.005720
LV2,-0.002927
LV3,0.002734
LV4,0.015454
LV5,0.027796


In [331]:
_module_tissue_data.squeeze().sort_values()

LV329   -0.084602
LV318   -0.073914
LV802   -0.071899
LV377   -0.068738
LV220   -0.068666
           ...   
LV927    0.069571
LV697    0.073310
LV550    0.074915
LV454    0.077556
LV744    0.086199
Name: I70-Diagnoses_main_ICD10_I70_Atherosclerosis, Length: 987, dtype: float64

In [332]:
lincs_projection.T[_drug_id].sort_values()

LV841   -0.086833
LV163   -0.080580
LV161   -0.070249
LV931   -0.069832
LV9     -0.069249
           ...   
LV974    0.053342
LV38     0.088232
LV743    0.097299
LV116    0.097558
LV881    0.154953
Name: DB00627, Length: 987, dtype: float64

In [333]:
_tmp = (-1.0 * lincs_projection.loc[_drug_id].dot(_module_tissue_data)).sort_values(ascending=False)
display(_tmp)

_top_phenomexcan_trait = _tmp.index[0]
display(_top_phenomexcan_trait)

I70-Diagnoses_main_ICD10_I70_Atherosclerosis    0.023
Name: DB00627, dtype: float64

'I70-Diagnoses_main_ICD10_I70_Atherosclerosis'

In [334]:
_tmp = lincs_projection.loc[_drug_id].multiply(_module_tissue_data[_top_phenomexcan_trait]).sort_values(ascending=True)
display(_tmp.head(20))

LV116   -0.006180
LV931   -0.004843
LV744   -0.002292
LV697   -0.002231
LV885   -0.001984
LV536   -0.001956
LV550   -0.001933
LV220   -0.001822
LV272   -0.001785
LV739   -0.001770
LV678   -0.001525
LV470   -0.001444
LV66    -0.001375
LV189   -0.001371
LV517   -0.001296
LV840   -0.001285
LV246   -0.001253
LV502   -0.001249
LV525   -0.001227
LV85    -0.001220
dtype: float64

In [335]:
",".join(_tmp.head(20).index)

'LV116,LV931,LV744,LV697,LV885,LV536,LV550,LV220,LV272,LV739,LV678,LV470,LV66,LV189,LV517,LV840,LV246,LV502,LV525,LV85'

In [336]:
_tmp2 = _tmp[_tmp < 0.0].sort_values(ascending=True)
display(_tmp2)
_tmp2.to_pickle("/tmp/lv_list.pkl")

LV116   -6.179892e-03
LV931   -4.842918e-03
LV744   -2.292495e-03
LV697   -2.230769e-03
LV885   -1.983519e-03
             ...     
LV574   -9.320074e-07
LV764   -8.293981e-07
LV683   -7.304520e-07
LV916   -9.235917e-08
LV877   -8.433177e-09
Length: 485, dtype: float64

In [337]:
_lv_name = "LV246"

In [338]:
_lv_genes = multiplier_z[_lv_name].sort_values(ascending=False).head(40)
display(_lv_genes)

SCD         6.672060
ACSS2       6.258514
GPAM        5.714077
DGAT2       4.738347
ACLY        3.708678
GPD1        3.346001
MVD         3.320598
FASN        2.915241
LPL         2.858278
ACSL1       2.682199
LSS         2.535996
FADS2       2.504681
AGPAT2      2.237759
ACACA       2.138277
ME1         2.107366
ELOVL5      2.022853
HSD17B12    2.007114
SLC7A10     1.968451
G0S2        1.900780
SREBF1      1.783389
CD36        1.763034
PC          1.744431
ELOVL3      1.676619
LPIN1       1.673466
CYB5A       1.641386
ELOVL6      1.617832
FZD4        1.612687
ACACB       1.549111
GBE1        1.539272
HMGCS1      1.507852
ACO1        1.480638
SLC25A1     1.478971
LPCAT3      1.475193
NSDHL       1.431425
HMGCR       1.398405
PCYT2       1.395157
LDLR        1.384271
IDI1        1.358324
NR1H3       1.321484
FAR2        1.291955
Name: LV246, dtype: float64

In [339]:
_module_tissue_data.loc[_lv_name, _top_phenomexcan_trait]

0.02983803260198739

In [340]:
lincs_projection.loc[_drug_id, _lv_name]#.sort_values()

-0.042009476075034115

In [341]:
_genes_assoc_data = _module_tissue_data_raw.reindex(
    _lv_genes.index
)[_top_phenomexcan_trait]

display(_genes_assoc_data)

SCD         1.526596
ACSS2       1.202443
GPAM        0.246954
DGAT2       1.380969
ACLY        0.091404
GPD1        0.695009
MVD        -0.262679
FASN             NaN
LPL        -1.235185
ACSL1      -0.426896
LSS        -0.693392
FADS2       0.077571
AGPAT2     -0.451378
ACACA      -2.451536
ME1        -1.239949
ELOVL5      0.563057
HSD17B12   -1.120733
SLC7A10    -0.543635
G0S2             NaN
SREBF1      1.678925
CD36             NaN
PC         -0.183990
ELOVL3     -1.032258
LPIN1       0.556787
CYB5A       1.149031
ELOVL6      0.948793
FZD4       -0.558263
ACACB      -0.901847
GBE1        0.496201
HMGCS1      0.484538
ACO1       -0.408778
SLC25A1    -0.074318
LPCAT3     -0.592774
NSDHL            NaN
HMGCR       1.513102
PCYT2       0.243787
LDLR       -0.509599
IDI1        0.924225
NR1H3       1.850190
FAR2             NaN
Name: I70-Diagnoses_main_ICD10_I70_Atherosclerosis, dtype: float64

In [342]:
_lincs_assoc_data = lincs_data.T.reindex(
    _lv_genes.index
)[_drug_id]

display(_lincs_assoc_data)

SCD        -1.288
ACSS2         NaN
GPAM          NaN
DGAT2         NaN
ACLY       -1.109
GPD1          NaN
MVD        -0.696
FASN       -0.328
LPL         1.305
ACSL1      -0.561
LSS        -0.453
FADS2      -0.460
AGPAT2     -0.679
ACACA      -0.006
ME1        -0.444
ELOVL5     -2.535
HSD17B12   -1.343
SLC7A10       NaN
G0S2       -1.228
SREBF1     -2.051
CD36        0.136
PC         -0.903
ELOVL3        NaN
LPIN1      -0.360
CYB5A      -0.341
ELOVL6      1.115
FZD4        0.869
ACACB      -0.741
GBE1       -0.648
HMGCS1     -1.916
ACO1       -0.719
SLC25A1     0.619
LPCAT3     -0.796
NSDHL         NaN
HMGCR       0.087
PCYT2      -0.553
LDLR       -3.322
IDI1       -0.887
NR1H3      -0.111
FAR2        0.717
Name: DB00627, dtype: float64

In [343]:
_genes_assoc_data.multiply(_lincs_assoc_data).sort_values(ascending=True)

SREBF1     -3.443475
SCD        -1.966255
LPL        -1.611916
ELOVL5     -1.427351
HMGCS1     -0.928374
IDI1       -0.819788
FZD4       -0.485131
CYB5A      -0.391819
GBE1       -0.321538
NR1H3      -0.205371
LPIN1      -0.200443
PCYT2      -0.134814
ACLY       -0.101367
SLC25A1    -0.046003
FADS2      -0.035683
ACACA       0.014709
HMGCR       0.131640
PC          0.166143
MVD         0.182825
ACSL1       0.239489
ACO1        0.293911
AGPAT2      0.306486
LSS         0.314107
LPCAT3      0.471848
ME1         0.550538
ACACB       0.668269
ELOVL6      1.057904
HSD17B12    1.505145
LDLR        1.692889
ACSS2            NaN
GPAM             NaN
DGAT2            NaN
GPD1             NaN
FASN             NaN
SLC7A10          NaN
G0S2             NaN
CD36             NaN
ELOVL3           NaN
NSDHL            NaN
FAR2             NaN
dtype: float64

In [346]:
# lincs_projection.loc[_drug_id].sort_values()

In [347]:
# _module_tissue_data.squeeze().sort_values().head(10)

### Gene-based - Genes driving association

In [135]:
_gene_based_tissue_data = pd.read_pickle(INPUT_DATA_DIR / "raw" / f"spredixcan-mashr-zscores-{_gene_based_tissue}-data.pkl")
_gene_based_tissue_data = _gene_based_tissue_data.rename(index=Gene.GENE_ID_TO_NAME_MAP)

display(_gene_based_tissue_data.shape)
display(_gene_based_tissue_data.head())

(16972, 4091)

,I9_PHLETHROMBDVTLOW-DVT_of_lower_extremities,I71-Diagnoses_main_ICD10_I71_Aortic_aneurysm_and_dissection,G62-Diagnoses_main_ICD10_G62_Other_polyneuropathies,2395_4-Hairbalding_pattern_Pattern_4,20003_1141168590-Treatmentmedication_code_pariet_10mg_ec_tablet,4056_raw-Age_stroke_diagnosed,M13_FOREIGNBODY-Residual_foreign_body_in_soft_tissue,20002_1494-Noncancer_illness_code_selfreported_varicose_veins,I85-Diagnoses_main_ICD10_I85_Oesophageal_varices,6158_4-Why_reduced_smoking_Financial_reasons,...,41215_0-Detention_categories_Informal_not_formally_detained,M76-Diagnoses_main_ICD10_M76_Enthesopathies_of_lower_limb_excluding_foot,22601_35633208-Job_coding_vocational_or_industrial_trainerinstructor,H7_RETINALDETACH-Retinal_detachments_and_breaks,20003_1141172918-Treatmentmedication_code_celluvisc_1_singleuse_eye_drops,22601_35643209-Job_coding_careers_adviser_or_vocational_guidance_specialist_placement_advisor,20002_1537-Noncancer_illness_code_selfreported_joint_pain,20514-Recent_lack_of_interest_or_pleasure_in_doing_things,22601_53192534-Job_coding_builder_building_contractor,30280_raw-Immature_reticulocyte_fraction
gene_id,,,,,,,,,,,,,,,,,,,,,
SCYL3,-3.380623,-0.668495,0.152390,1.973963,-1.662351,-0.392219,-0.469066,-1.096786,-1.001884,-0.718271,...,-0.608140,0.169738,0.030607,0.402069,0.311547,0.844089,0.329361,-2.067595,1.963715,-0.017090
C1orf112,-3.310312,-0.802099,0.509401,2.057412,-1.181577,-0.627847,-0.402729,-1.119259,-1.037078,-0.640086,...,-0.405886,0.278882,0.135687,0.059612,0.376847,0.900322,0.550222,-1.953270,2.162665,-0.433549
FGR,1.917626,0.208054,-0.294232,-0.706293,1.390585,-0.111784,0.124277,1.135089,0.097681,-1.829892,...,-0.485489,-0.177272,0.007996,0.934915,-0.660882,-0.685404,0.587479,-0.313904,1.020958,0.290251
CFH,0.466666,-0.091782,0.870694,0.628786,-0.472838,-0.519768,2.424860,-1.551633,-0.287467,-0.867953,...,-0.934352,0.300906,-0.619276,0.197975,-0.647014,0.715572,-0.713293,-2.474839,-2.248581,0.460525
FUCA2,-0.653523,0.791157,0.215262,0.717572,-2.534597,0.284291,-0.411587,0.228705,0.853281,-0.436868,...,-2.085503,-1.295726,0.619484,0.702873,1.450982,-0.728286,-0.448195,2.412672,-1.695031,-0.928210


In [136]:
_gene_based_tissue_data.index.is_unique

False

In [137]:
_gene_based_tissue_data.index[_gene_based_tissue_data.index.duplicated(keep="first")]

Index(['SPATA13', 'LINC01422', 'LINC00484', 'LINC01115', 'LYNX1'], dtype='object', name='gene_id')

In [138]:
_gene_based_tissue_data = _gene_based_tissue_data.loc[
    ~_gene_based_tissue_data.index.duplicated(keep="first")
]

In [139]:
_gene_based_tissue_data.shape

(16967, 4091)

In [140]:
_gene_based_tissue_data.index.is_unique

True

In [75]:
# from drug_disease import _zero_nontop_genes

In [76]:
OUTPUT_PREDICTIONS_DIR

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg')

In [77]:
# _prefix = "top_5_genes"

_all_dfs = []

for ntc in (None, 50, 100, 250, 500):
    _prefix = "all_genes"
    if ntc is not None:
        _prefix = f"top_{ntc}_genes"
    
    _df = pd.read_hdf(
        OUTPUT_PREDICTIONS_DIR / f"spredixcan-mashr-zscores-{_gene_based_tissue}-data-{_prefix}-prediction_scores.h5",
        key="prediction"
    )
    
    _score_col_name = f"score_{_prefix}"
    _df = pd.merge(
        _df, gold_standard, on=["trait", "drug"], how="inner"
    ).drop(columns=["true_class"]).set_index(["trait", "drug"]).rename(columns={"score": _score_col_name})
    
    _df[_score_col_name] = _df[_score_col_name].rank()
    
    _all_dfs.append(_df)

In [78]:
_df

score_top_500_genes
trait        drug                        
DOID:0050741 DB00215                311.0
             DB00704                503.0
             DB00822                477.0
DOID:10283   DB00014                 52.0
             DB00175                 77.0
...                                   ...
DOID:9970    DB00397                 45.0
             DB00501                 80.0
             DB01083                 83.0
             DB01105                183.0
             DB01156                 26.0

[646 rows x 1 columns]

In [79]:
_raw_predictions_tissue = pd.concat(_all_dfs, axis=1, join="inner")#.mean(axis=1)

In [80]:
_raw_predictions_tissue.shape

(646, 5)

In [81]:
_raw_predictions_tissue.head()

score_all_genes  score_top_50_genes  \
trait        drug                                           
DOID:0050741 DB00215            303.0               260.0   
             DB00704            482.0               444.0   
             DB00822            437.0               256.0   
DOID:10283   DB00014             58.0                62.0   
             DB00175             44.0                65.0   

                      score_top_100_genes  score_top_250_genes  \
trait        drug                                                
DOID:0050741 DB00215                181.0                190.0   
             DB00704                396.0                482.0   
             DB00822                459.0                255.0   
DOID:10283   DB00014                 69.0                 45.0   
             DB00175                 71.0                 76.0   

                      score_top_500_genes  
trait        drug                          
DOID:0050741 DB00215                311.0  
             DB00704                503.0  
             DB00822                477.0  
DOID:10283   DB00014                 52.0  
             DB00175                 77.0

In [82]:
_raw_predictions_tissue = _raw_predictions_tissue.mean(axis=1)

In [83]:
display(_raw_predictions_tissue.shape)
display(_raw_predictions_tissue.head())

assert _df.shape[0] == _raw_predictions_tissue.shape[0]
assert not _raw_predictions_tissue.isna().any().any()

(646,)

trait         drug   
DOID:0050741  DB00215    249.0
              DB00704    461.4
              DB00822    376.8
DOID:10283    DB00014     57.2
              DB00175     66.6
dtype: float64

In [85]:
_raw_predictions_tissue.loc[_doid, _drug_id]# .query(f"(trait == '{_doid}') and (drug == '{_drug_id}')")

512.6

In [ ]:
# FIXME: add assert here to make sure this number above is the same in predictions_avg

In [141]:
# _traits_proj = pd.read_pickle(INPUT_DATA_DIR / "proj" / f"spredixcan-mashr-zscores-{_module_tissue}-projection.pkl")

_gene_based_tissue_data = _gene_based_tissue_data[
    _phenomexcan_traits
#     "C62-Diagnoses_main_ICD10_C62_Malignant_neoplasm_of_testis",
#     "I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease",
]

# _traits_proj = _traits_proj.apply(
#     lambda x: _zero_nontop_genes(x, 50, use_abs=True)
# )

In [143]:
_gene_based_tissue_data.head()

,I70-Diagnoses_main_ICD10_I70_Atherosclerosis
gene_id,
SCYL3,-1.106146
C1orf112,-1.258885
FGR,-1.781236
CFH,-0.233247
FUCA2,0.801598


In [144]:
_gene_based_tissue_data.squeeze().sort_values()

gene_id
NMRK1          -4.415421
RP11-54O7.18   -4.065244
OSTF1          -3.979124
RP5-916L7.2    -3.772428
AGRN           -3.685653
                  ...   
SEMA4C          3.597588
MTF1            3.777043
FAM178B         4.457912
SEC14L1         4.542709
LPA             4.872263
Name: I70-Diagnoses_main_ICD10_I70_Atherosclerosis, Length: 16967, dtype: float64

In [146]:
lincs_data.T[_drug_id].sort_values()

TAGLN    -6.716
IFRD2    -5.166
S100P    -4.709
LONP1    -4.120
ACADVL   -3.765
          ...  
OLFM1     3.665
FZD1      3.934
PPBP      4.168
HERC6     4.233
WIF1      4.620
Name: DB00627, Length: 7120, dtype: float64

In [152]:
# get common genes with lincs
common_genes = _gene_based_tissue_data.index.intersection(lincs_data.columns)
_gene_data = _gene_based_tissue_data.loc[common_genes]
_lincs_data = lincs_data[common_genes]

In [153]:
_gene_data.shape

(5767, 1)

In [154]:
_lincs_data.shape

(1170, 5767)

In [170]:
_tmp = (-1.0 * _lincs_data.loc[_drug_id].dot(_gene_data)).sort_values(ascending=False)
display(_tmp)

_top_phenomexcan_trait = _tmp.index[0]
display(_top_phenomexcan_trait)

I70-Diagnoses_main_ICD10_I70_Atherosclerosis    191.118501
Name: DB00627, dtype: float64

'I70-Diagnoses_main_ICD10_I70_Atherosclerosis'

In [171]:
_tmp = _lincs_data.loc[_drug_id].multiply(_gene_data[_top_phenomexcan_trait]).sort_values(ascending=True)
display(_tmp.head(20))

IFNAR1    -6.731023
NT5DC2    -6.705433
TRIM22    -6.552178
HEBP1     -6.430294
R3HDM4    -6.119445
SYK       -6.047916
PRR15L    -5.998544
CAMSAP2   -5.853971
USP18     -5.402533
N6AMT1    -5.391901
ISOC1     -5.330889
PRKCD     -5.312601
NMRK1     -5.179289
MTF1      -4.827061
MICA      -4.594171
CLTB      -4.561361
ASL       -4.539134
SF3A3     -4.416862
IFIT1     -4.357457
TRMT61B   -4.341936
dtype: float64

In [172]:
",".join(_tmp.head(20).index)

'IFNAR1,NT5DC2,TRIM22,HEBP1,R3HDM4,SYK,PRR15L,CAMSAP2,USP18,N6AMT1,ISOC1,PRKCD,NMRK1,MTF1,MICA,CLTB,ASL,SF3A3,IFIT1,TRMT61B'

In [173]:
_tmp[_tmp < 0.0].shape

(2930,)

In [174]:
_lincs_data.loc[_drug_id].sort_values()

TAGLN    -6.716
S100P    -4.709
LONP1    -4.120
ACADVL   -3.765
VAT1     -3.698
          ...  
PRR15L    3.606
FGFR2     3.648
OLFM1     3.665
HERC6     4.233
WIF1      4.620
Name: DB00627, Length: 5767, dtype: float64

In [179]:
_lincs_data.loc[_drug_id, "NT5DC2"]#.sort_values()

-2.529

In [176]:
_gene_data.squeeze().sort_values()

NMRK1      -4.415421
OSTF1      -3.979124
AGRN       -3.685653
PDHB       -3.597112
PLA2G16    -3.450018
              ...   
NUPL2       3.417509
KIAA0040    3.588833
SEMA4C      3.597588
MTF1        3.777043
SEC14L1     4.542709
Name: I70-Diagnoses_main_ICD10_I70_Atherosclerosis, Length: 5767, dtype: float64

In [181]:
_gene_data.squeeze().loc["NT5DC2"]

2.651416778564453

## hypertension

In [73]:
find_differences("hypertension")

disease                  drug        method  \
doid_id    drugbank_id                                                     
DOID:10763 DB01136      hypertension            Carvedilol  Module-based   
           DB01136      hypertension            Carvedilol    Gene-based   
           DB00820      hypertension             Tadalafil  Module-based   
           DB00820      hypertension             Tadalafil    Gene-based   
           DB01001      hypertension            Salbutamol  Module-based   
           DB01001      hypertension            Salbutamol    Gene-based   
           DB00758      hypertension           Clopidogrel  Module-based   
           DB00758      hypertension           Clopidogrel    Gene-based   
           DB00598      hypertension             Labetalol  Module-based   
           DB00598      hypertension             Labetalol    Gene-based   
           DB00521      hypertension             Carteolol  Module-based   
           DB00521      hypertension             Carteolol    Gene-based   
           DB01194      hypertension          Brinzolamide  Module-based   
           DB01194      hypertension          Brinzolamide    Gene-based   
           DB00384      hypertension           Triamterene  Module-based   
           DB00384      hypertension           Triamterene    Gene-based   
           DB00374      hypertension          Treprostinil  Module-based   
           DB00374      hypertension          Treprostinil    Gene-based   
           DB00350      hypertension             Minoxidil  Module-based   
           DB00350      hypertension             Minoxidil    Gene-based   
           DB01203      hypertension               Nadolol  Module-based   
           DB01203      hypertension               Nadolol    Gene-based   
           DB00524      hypertension            Metolazone  Module-based   
           DB00524      hypertension            Metolazone    Gene-based   
           DB00887      hypertension            Bumetanide  Module-based   
           DB00887      hypertension            Bumetanide    Gene-based   
           DB01083      hypertension              Orlistat  Module-based   
           DB01083      hypertension              Orlistat    Gene-based   
           DB01023      hypertension            Felodipine  Module-based   
           DB01023      hypertension            Felodipine    Gene-based   
           DB00905      hypertension           Bimatoprost  Module-based   
           DB00905      hypertension           Bimatoprost    Gene-based   
           DB01244      hypertension              Bepridil  Module-based   
           DB01244      hypertension              Bepridil    Gene-based   
           DB00177      hypertension             Valsartan  Module-based   
           DB00177      hypertension             Valsartan    Gene-based   
           DB00722      hypertension            Lisinopril  Module-based   
           DB00722      hypertension            Lisinopril    Gene-based   
           DB00457      hypertension              Prazosin  Module-based   
           DB00457      hypertension              Prazosin    Gene-based   
           DB00187      hypertension               Esmolol  Module-based   
           DB00187      hypertension               Esmolol    Gene-based   
           DB00436      hypertension   Bendroflumethiazide  Module-based   
           DB00436      hypertension   Bendroflumethiazide    Gene-based   
           DB00492      hypertension            Fosinopril  Module-based   
           DB00492      hypertension            Fosinopril    Gene-based   
           DB00206      hypertension             Reserpine  Module-based   
           DB00206      hypertension             Reserpine    Gene-based   
           DB00774      hypertension    Hydroflumethiazide  Module-based   
           DB00774      hypertension    Hydroflumethiazide    Gene-based   
           DB00264      hypertension            Metoprolol  Module-based   
 

## allergic rhinitis

In [70]:
find_differences("allergic rhinitis")

disease                drug        method  \
doid_id   drugbank_id                                                        
DOID:4481 DB00443      allergic rhinitis       Betamethasone    Gene-based   
          DB00443      allergic rhinitis       Betamethasone  Module-based   
          DB01234      allergic rhinitis       Dexamethasone    Gene-based   
          DB01234      allergic rhinitis       Dexamethasone  Module-based   
          DB00741      allergic rhinitis      Hydrocortisone    Gene-based   
          DB00741      allergic rhinitis      Hydrocortisone  Module-based   
          DB00959      allergic rhinitis  Methylprednisolone    Gene-based   
          DB00959      allergic rhinitis  Methylprednisolone  Module-based   
          DB00620      allergic rhinitis       Triamcinolone    Gene-based   
          DB00620      allergic rhinitis       Triamcinolone  Module-based   
          DB00180      allergic rhinitis         Flunisolide    Gene-based   
          DB00180      allergic rhinitis         Flunisolide  Module-based   
          DB00860      allergic rhinitis        Prednisolone    Gene-based   
          DB00860      allergic rhinitis        Prednisolone  Module-based   

                       score  true_class  score_zscore  different_sign  \
doid_id   drugbank_id                                                    
DOID:4481 DB00443      472.7         1.0     -0.536777            True   
          DB00443      618.7         1.0      1.201407            True   
          DB01234      472.7         1.0     -0.536777            True   
          DB01234      618.7         1.0      1.201407            True   
          DB00741      394.2         1.0     -1.415158            True   
          DB00741      551.8         1.0      0.290861            True   
          DB00959      411.0         1.0     -1.227173            True   
          DB00959      538.4         1.0      0.108480            True   
          DB00620      484.2         1.0     -0.408097            True   
          DB00620      574.2         1.0      0.595738            True   
          DB00180      509.0         1.0     -0.130596            True   
          DB00180      563.6         1.0      0.451466            True   
          DB00860      527.8         1.0      0.079768            True   
          DB00860      503.8         1.0     -0.362445            True   

                       score_difference  
doid_id   drugbank_id                    
DOID:4481 DB00443              1.738184  
          DB00443              1.738184  
          DB01234              1.738184  
          DB01234              1.738184  
          DB00741              1.706019  
          DB00741              1.706019  
          DB00959              1.335653  
          DB00959              1.335653  
          DB00620              1.003835  
          DB00620              1.003835  
          DB00180              0.582061  
          DB00180              0.582061  
          DB00860              0.442213  
          DB00860              0.442213

# Alzheimer's disease

In [31]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "max_colwidth", None
):
    _tmp = (
        pharmadb_predictions[
            pharmadb_predictions["disease"].isin(["allergic rhinitis"])
        ]
        #         .query("method == 'Module-based'")
        .sort_values(["score_zscore"], ascending=[False])
    )
    display(_tmp)

,doid_id,drugbank_id,disease,drug,method,score,true_class,score_zscore
61,DOID:4481,DB01234,allergic rhinitis,Dexamethasone,Module-based,618.7,1.0,1.201407
53,DOID:4481,DB00443,allergic rhinitis,Betamethasone,Module-based,618.7,1.0,1.201407
57,DOID:4481,DB00434,allergic rhinitis,Cyproheptadine,Module-based,578.0,1.0,0.647458
56,DOID:4481,DB00434,allergic rhinitis,Cyproheptadine,Gene-based,576.8,1.0,0.628057
77,DOID:4481,DB00620,allergic rhinitis,Triamcinolone,Module-based,574.2,1.0,0.595738
54,DOID:4481,DB00341,allergic rhinitis,Cetirizine,Gene-based,562.6,1.0,0.469165
63,DOID:4481,DB00180,allergic rhinitis,Flunisolide,Module-based,563.6,1.0,0.451466
72,DOID:4481,DB00768,allergic rhinitis,Olopatadine,Gene-based,556.4,1.0,0.399790
73,DOID:4481,DB00768,allergic rhinitis,Olopatadine,Module-based,556.2,1.0,0.350748
55,DOID:4481,DB00341,allergic rhinitis,Cetirizine,Module-based,552.6,1.0,0.301750
